In [2]:
%pip install jsonlines

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import jsonlines
import json
import os
import numpy as np

1. box_output.manifest 는 오토바이 box를 라벨링한 manifest 파일
2. moto_output.manifest 는 오토바이를 라벨링한 manifest 파일
3. output.manifest 는 1,2번의 annotation을 합친 파일
4. train.manifest 는 3번을 랜덤하게 재배열한 뒤 80%를 잘라낸 파일
5. valid.manifest 는 3번을 랜덤하게 재배열한 뒤 20%를 잘라낸 파일
6. _labeling_key 는 label 작업명입니다.
    manifest파일내부의 key 값을 확인 후 수정해야합니다.

## {"source-ref":"s3://sagemaker-motorcycle-test/modified_data/image_batch/RQ1919553563_01.jpg","v2-delivery-box-labeling":{"image_size":[{"width":1280,"height":960,"depth":3}],"annotations":[{"class_id":1,"top":4,"left":509,"height":427,"width":497}]},"v2-delivery-box-labeling-metadata":{"objects":[{"confidence":0.09}],"class-map":{"0":"delivery box"},"type":"groundtruth/object-detection","human-annotated":"yes","creation-date":"2020-09-22T06:27:53.293356","job-name":"labeling-job/v2-delivery-box-labeling"}}

In [5]:
_manifest_path = 'manifest'
_out_manifest_box = os.path.join(os.getcwd(),_manifest_path,'box_output.manifest')
_out_manifest_moto = os.path.join(os.getcwd(),_manifest_path,'moto_output.manifest')
_new_out_manifest = os.path.join(os.getcwd(),_manifest_path,'output.manifest')
_augmented_manifest_filename_train = os.path.join(os.getcwd(),_manifest_path,'train.manifest')
_augmented_manifest_filename_valid = os.path.join(os.getcwd(),_manifest_path,'valid.manifest')
_labeling_key = 'v2-delivery-box-labeling'

In [6]:
'''
Json Line 포맷 파일을 읽고/쓰기
'''
def getJsonLines(fname):
    with jsonlines.open(fname, 'r') as reader:
        return list(reader)

def createJsonLineFile(fname, data):
    with open(fname,'w') as f:
        for line in data:
            f.write(json.dumps(line))
            f.write('\n')

## {"source-ref":"s3://sagemaker-motorcycle-test/modified_data/image_batch/RQ1919553563_01.jpg","v2-delivery-box-labeling":{"image_size":[{"width":1280,"height":960,"depth":3}],"annotations":[{"class_id":1,"top":4,"left":509,"height":427,"width":497}]},"v2-delivery-box-labeling-metadata":{"objects":[{"confidence":0.09}],"class-map":{"0":"delivery box"},"type":"groundtruth/object-detection","human-annotated":"yes","creation-date":"2020-09-22T06:27:53.293356","job-name":"labeling-job/v2-delivery-box-labeling"}}

In [7]:
'''
Annotations Array 가져오기
'''
def findAnnotations(lists,ref):
    for i, data in enumerate(lists):
        if (data['source-ref'] == ref):
            ##print(data['v2-delivery-box-labeling']['annotations'])
            return data['v2-delivery-box-labeling']['annotations']
    return []

'''
Annotations Array 에서 Class 정보 가져오기 
'''
def findAnnotation(lists,no):
    for i, data in enumerate(lists):
        if (data['class_id'] == no):
            return data
    return None

'''
두개의 manifest 파일을 합치는 기능
file2의 class 1의 annotation 값을 file1에 추가합니다.
'''
def mergeJson(file1, file2):

    box_lists = getJsonLines(file1)
    moto_lists = getJsonLines(file2)

    for i, data in enumerate(box_lists):
        moto_dict = findAnnotation(findAnnotations(moto_lists,data['source-ref']),1)
        if (moto_dict != None):
            data[_labeling_key]['annotations'].append(moto_dict)
            data[f'{_labeling_key}-metadata']['class-map'] =  {"0": "delivery box", "1":"motocycle"}
            data[f'{_labeling_key}-metadata']['objects'].append({"confidence":0.09})

    createJsonLineFile(_new_out_manifest, box_lists)
    makeSplitJson(box_lists)

'''
Array에서 train 데이터와 validation 데이터 나누기
'''

def makeSplitJson(lines):

    print(f'total count : {len(lines)}')
    np.random.shuffle(lines)
    dataset_size = len(lines)
    num_train_samples = round(dataset_size * 0.8)

    train_data = lines[:num_train_samples]
    validation_data = lines[num_train_samples:]

    print(f'train count : {len(train_data)}')
    print(f'validation count :{len(validation_data)}')
    createJsonLineFile(_new_out_manifest,lines)
    createJsonLineFile(_augmented_manifest_filename_train,train_data)
    createJsonLineFile(_augmented_manifest_filename_valid,validation_data)

def main():
    mergeJson(_out_manifest_box, _out_manifest_moto)

두개의 output 파일을 합치기

In [9]:
main()

total count : 2662
train count : 2130
validation count :532
